# Gradient-Effected Object Recognition Gauge for hive Entrances (GEORGE)
Neural-net-powered honeybee hive-mounted pollen, varroa, and wasp counter

## Imports

In [1]:
import sys
sys.path.append("/home/kevinhardin/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games")
sys.path.append("CameraScript")
sys.path.append("/home/kevinhardin/Documents/GEORGE")
from CameraScript import takePic
from time import sleep, time
from datetime import datetime, date
import os
os.environ["LIBCAMERA_LOG_LEVELS"] = "2"
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import glob
import numpy as np
import tensorflow as tf
from pathlib import Path
from GEORGE_Library import *
%matplotlib inline

print("Done")

Done


## Define global variables

In [2]:
label_id_offset = 1
score_threshold = 0.8
category_index = {1: {'id': 1, 'name': 'regular'}, 2: {'id': 2, 'name': 'pollen'}, 3: {'id': 3, 'name': 'varroa'}, 4: {'id': 4, 'name': 'wasps'}}
model_directory = '/home/kevinhardin/Documents/GEORGE'
model_name = 'extract_superimp_model'
model_dest = os.path.join(os.sep, model_directory, model_name)

print("Done")

Done


In [3]:
new_model = tf.saved_model.load(model_dest)

plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [15, 15]

save_plot = False

im, _, _ = takePic()

for i in range(3):
    image = np.array(im).astype('uint8')[440:, (640 * i):(640 * (i + 1)), :]
    ax = plt.subplot(1, 3, i + 1)

    bboxes = []
    class_ids = []
    scores = []

    input_tensor = tf.convert_to_tensor(image, dtype=tf.float32)
    input_tensor = tf.expand_dims(input_tensor, 0)
    detections = new_model.signatures['detect'](input_tensor)

    for k in range(len(detections['detection_boxes'][0])):
        if detections['detection_scores'][0][k].numpy() >= score_threshold:
            y1, x1, y2, x2  = np.array(detections['detection_boxes'][0][k])
            bboxes.append([y1, x1, y2, x2])
            class_id = detections['detection_classes'][0][k].numpy().astype(np.uint32) + label_id_offset
            class_name = category_index[class_id]['name']
            score = detections['detection_scores'][0][k].numpy()
            class_ids.append(class_id)
            scores.append(score)

    if not scores == []:
        plot_detections(
        image,
        np.asarray(bboxes),
        np.asarray(class_ids),
        scores,
        category_index)#, figsize=(30, 15))
                  
plt.suptitle('GEORGE Validation (Score Thresh: %s)' % (score_threshold), fontsize=20)
if save_plot:
    plt.savefig('GEORGE_Validation_Plot.png')
plt.show()

IndexError: Read less bytes than requested